# Выбор локации для скважины

**Данные:** пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.

**Описание данных:**

id — уникальный идентификатор скважины;<br>
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);<br>
product — объём запасов в скважине (тыс. баррелей).<br>

**Цель исследования:** построить модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой Bootstrap.

**Исследование пройдёт в несколько этапов:**

Импорт библиотек.<br>
Загрузка и подготовка данных.<br>
Разбивка данных на выборки.<br>
Обучение моделей и расчет метрики RMSE.<br>
Расчет суммарной прибыли по регионам.<br>
Техника Bootstrap для проверки распределения прибыли.<br>
Расчет средней прибыли, 95%-го доверительного интервала. <br>
Расчет риска убытков.<br> 
Вывод.<br>

**Задача будет решаться с помощью модели линейной регрессии.**

## Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import stats as st

## Загрузка и подготовка данных

### Загрузка данных и просмотр информации о них

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data_1.shape[0] #Размер первой выборки

100000

In [4]:
data_1.dtypes #проверка типов данных

id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

In [5]:
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_2.head() 

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_2.shape[0] #Размер второй выборки

100000

In [7]:
data_2.dtypes #проверка типов данных

id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

In [8]:
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
data_3.head() 

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
data_3.shape[0] #Размер третьей выборки

100000

In [10]:
data_3.dtypes #проверка типов данных

id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

Промежуточный вывод:

- надо проверить данные на пропуски,
- надо проверить данные на дубликаты.

С типами данных все в порядке, размеры выборок одинаковые - 100000 строк.

### Проверка на пропуски

In [11]:
data_1.isnull().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [12]:
data_2.isnull().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [13]:
data_3.isnull().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Промежуточный вывод:

- пропусков в данных нет.

### Проверка на дубликаты

In [14]:
data_1.duplicated().sum() #Проверка на дубликаты

0

In [15]:
data_2.duplicated().sum() #Проверка на дубликаты

0

In [16]:
data_3.duplicated().sum() #Проверка на дубликаты

0

Промежуточный вывод:

- дубликатов в данных нет.

### Разбивка данных на выборки

Разобью данные на тренировочную и валидационную выборку в соотношении 75:25.
Также я решила из features удалить столбец с ID - он никак не поможет обучить модель.

In [17]:
target_1 = data_1['product']
features_1 = data_1.drop(['product', 'id'], axis='columns')

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

print(features_train_1.shape[0], features_valid_1.shape[0]) #проверка размера выборок
print(target_train_1.shape[0], target_valid_1.shape[0])

75000 25000
75000 25000


In [18]:
target_2 = data_2['product']
features_2 = data_2.drop(['product', 'id'], axis='columns')

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

print(features_train_2.shape[0], features_valid_2.shape[0]) #проверка размера выборок
print(target_train_2.shape[0], target_valid_2.shape[0])

75000 25000
75000 25000


In [19]:
target_3 = data_3['product']
features_3 = data_3.drop(['product', 'id'], axis='columns')

features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=12345)

print(features_train_3.shape[0], features_valid_3.shape[0]) #проверка размера выборок
print(target_train_3.shape[0], target_valid_3.shape[0])

75000 25000
75000 25000


Вывод:

- проведена проверка на пропуски, их нет
- проведена проверка на дубли, их нет
- данные разбиты на выборки в соотношении 75:25.

## Обучение и проверка модели

### Обучение модели на данных из первого региона:

При расчете предсказаний сразу же перевожу их в формат Series и оставляю индексы скважин, чтобы потом все работало корректно.

In [20]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = pd.Series(model_1.predict(features_valid_1), index=features_valid_1.index)

rmse_1 = (mean_squared_error(target_valid_1, predicted_valid_1))** 0.5
print("RMSE первого региона =", rmse_1)
print('Средний запас предсказанного сырья первого региона =', predicted_valid_1.mean())

RMSE первого региона = 37.5794217150813
Средний запас предсказанного сырья первого региона = 92.59256778438035


### Обучение модели на данных из второго региона:

In [21]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = pd.Series(model_2.predict(features_valid_2), index=features_valid_2.index)

rmse_2 = (mean_squared_error(target_valid_2, predicted_valid_2))** 0.5
print("RMSE второго региона =", rmse_2)
print('Средний запас предсказанного сырья второго региона =', predicted_valid_2.mean())

RMSE второго региона = 0.893099286775617
Средний запас предсказанного сырья второго региона = 68.728546895446


### Обучение модели на данных из третьего региона:

In [22]:
model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predicted_valid_3 = pd.Series(model_3.predict(features_valid_3), index=features_valid_3.index)

rmse_3 = (mean_squared_error(target_valid_3, predicted_valid_3))** 0.5
print("RMSE третьего региона =", rmse_3)
print('Средний запас предсказанного сырья третьего региона =', predicted_valid_3.mean())

RMSE третьего региона = 40.02970873393434
Средний запас предсказанного сырья третьего региона = 94.96504596800489


**Промежуточный вывод:**

Самый большой запас предсказанного сырья в третьем регионе - 95 тыс. баррелей, но метрика RMSE далека от идеала, самая лучшая метрика у модели по второму региону - 0.893. Значит, ошибка по расчетам этих данных самая минимальная. Поэтому более надежна вторая модель.

## Подготовка к расчёту прибыли

In [23]:
budget = 10000000000 #сколько надо денег в рублях на разработку скважиин
income_per_thousand = 450000 #сколько в рублях приносит тысяча баррелей

#ожидаемый объем сырья для безубыточной стратегии
expected_profit = budget / income_per_thousand / 200
print('Достаточный объём сырья для безубыточной разработки новой скважины = ', expected_profit, 'тыс.баррелей')

Достаточный объём сырья для безубыточной разработки новой скважины =  111.11111111111111 тыс.баррелей


**Промежуточный вывод:**

Во всех 3 регионах минимальное среднее предсказанное количество сырья меньше, чем требуется для безубыточной стратегии при бурении скважины. Но нужна перепроверка с помощью бутстрэпа, так как это очень простые расчеты, которые не всегда корректно отображают результат.

## Расчёт прибыли и рисков 

### Формула для расчета прибыли:

Для расчета прибыли я создам функию и применю ее к датасетам трех регионов, чтобы увидеть суммарную прибыль по топ-200 скважинам по запасам сырья.

In [24]:
def revenue(target, predictions, count):
    predictions_sorted = predictions.sort_values(ascending=False) #сортировка предсказаний от большего к меньшему
    valid_indexes = predictions_sorted.iloc[0:count].index #индексы первых топ объектов по count
    selected = target.loc[valid_indexes] #выбор строк таргета с номерами индексов из предсказаний
    #расчет прибыли: цена за тысячу, умноженная на сумму объема объектов и от этого числа отнимается бюджет
    return (income_per_thousand * selected.sum()) - budget 

### Прибыль первого региона:

In [25]:
revenue_1 = revenue(target_valid_1, predicted_valid_1, 200) #считаю прибыль по топ-200 объектам
print('Прибыль в первом регионе:', round(revenue_1, 2))

Прибыль в первом регионе: 3320826043.14


### Прибыль второго региона:

In [26]:
revenue_2 = revenue(target_valid_2, predicted_valid_2, 200) #считаю прибыль по топ-200 объектам
print('Прибыль во втором регионе:', round(revenue_2, 2))

Прибыль во втором регионе: 2415086696.68


### Прибыль третьего региона:

In [27]:
revenue_3 = revenue(target_valid_3, predicted_valid_3, 200) #считаю прибыль по топ-200 объектам
print('Прибыль в третьем регионе:', round(revenue_3, 2))

Прибыль в третьем регионе: 2710349963.6


**Промежуточный вывод:**

По расчету самый прибыльный первый регион, на втором месте третий регион.

## Bootstrap для распределения прибыли:

Применю технику bootstrap для трех датасетов и рассчитаю доверительный интервал 95% и риски убытков:

### Bootstrap для распределения прибыли первого региона:

In [28]:
state = np.random.RandomState(12345)
values_1 = []

for i in range(1000): #добавляю 1000 значений прибыли техникой бутстрэп
    target_sample_1 = target_valid_1.sample(n=500, replace=True, random_state=state)
    valid_indexes_1 = target_sample_1.iloc[0:500].index
    predictions_1 = predicted_valid_1.loc[valid_indexes_1]
    values_1.append(revenue(target_sample_1, predictions_1, 200))
    
values_1 = pd.Series(values_1) #перевожу список в Series

mean_1 = values_1.mean() #среднее по прибыли
print("Средняя прибыль первого региона:", mean_1)

confidence_interval_1 = st.t.interval(
    0.95, len(values_1)-1, values_1.mean(), values_1.sem()) #доверительный интервал

print("95%-ый доверительный интервал первого региона:", confidence_interval_1)

#риск убытков:
risk_of_loss_1 = 0
for i in range(1000):
    if values_1[i] < 0:
        risk_of_loss_1 +=1
        
risk_of_loss_1 = risk_of_loss_1 / 1000

print("Риск убытков первого региона:", risk_of_loss_1)

Средняя прибыль первого региона: 600735244.2611653
95%-ый доверительный интервал первого региона: (580991087.3188789, 620479401.2034516)
Риск убытков первого региона: 0.02


### Bootstrap для распределения прибыли второго региона:

In [30]:
values_2 = []

for i in range(1000): #добавляю 1000 значений прибыли техникой бутстрэп
    target_sample_2 = target_valid_2.sample(n=500, replace=True, random_state=state)
    valid_indexes_2 = target_sample_2.iloc[0:500].index
    predictions_2 = predicted_valid_2.loc[valid_indexes_2]
    values_2.append(revenue(target_sample_2, predictions_2, 200))
    
values_2 = pd.Series(values_2) #перевожу список в Series

mean_2 = values_2.mean() #среднее по прибыли
print("Средняя прибыль:", mean_2)

confidence_interval_2 = st.t.interval(
    0.95, len(values_2)-1, values_2.mean(), values_2.sem()) #доверительный интервал

print("95%-ый доверительный интервал:", confidence_interval_2)

#риск убытков:
risk_of_loss_2 = 0
for i in range(1000):
    if values_2[i] < 0:
        risk_of_loss_2 +=1
        
risk_of_loss_2= risk_of_loss_2 / 1000

print("Риск убытков:", risk_of_loss_2)

Средняя прибыль: 663958995.2601906
95%-ый доверительный интервал: (648077906.0159814, 679840084.5043998)
Риск убытков: 0.001


### Bootstrap для распределения прибыли третьего региона:

In [31]:
values_3 = []

for i in range(1000): #добавляю 1000 значений прибыли техникой бутстрэп
    target_sample_3 = target_valid_3.sample(n=500, replace=True, random_state=state)
    valid_indexes_3 = target_sample_3.iloc[0:500].index
    predictions_3 = predicted_valid_3.loc[valid_indexes_3]
    values_3.append(revenue(target_sample_3, predictions_3, 200))
    
values_3 = pd.Series(values_3) #перевожу список в Series

mean_3 = values_3.mean() #среднее по прибыли
print("Средняя прибыль:", mean_3)

confidence_interval_3 = st.t.interval(
    0.95, len(values_3)-1, values_3.mean(), values_3.sem()) #доверительный интервал

print("95%-ый доверительный интервал:", confidence_interval_3)

#риск убытков:
risk_of_loss_3 = 0
for i in range(1000):
    if values_3[i] < 0:
        risk_of_loss_3 +=1
        
risk_of_loss_3 = risk_of_loss_3 / 1000

print("Риск убытков:", risk_of_loss_3)

Средняя прибыль: 597381047.9005232
95%-ый доверительный интервал: (577080076.0794092, 617682019.7216371)
Риск убытков: 0.025


**Промежуточный вывод:**

Судя по расчетам, выгоднее всего бурить новую скважину во втором регионе, так как меньше всего риск убытков и самая большая прибыль.

## **Общий вывод:**

При работе над задачей была выполнена проверка данных. Данные были разбиты на выборки для машинного обучения в соотношении 75:25.

Также была обучена модель линейной регрессии по 3 датасетам, получены предсказания по объему сырья, также рассчитана метрика RMSE:

-------------------------------------------------------------------------------------------------------------------------------

**RMSE первого региона:** 37.579<br>
**Средний запас предсказанного сырья первого региона:** 92.592<br>
<br>
**RMSE второго региона:** 0.893<br>
**Средний запас предсказанного сырья второго региона:** 68.728<br>
<br>
**RMSE второго региона:** 40.029<br>
**Средний запас предсказанного сырья третьего региона:** 94.965<br>

-------------------------------------------------------------------------------------------------------------------------------

Самый большой запас предсказанного сырья в третьем регионе, но метрика RMSE далека от идеала, самая лучшая метрика у модели по второму региону. Значит, ошибка по расчетам этих данных самая минимальная. Поэтому более надежна вторая модель.

-------------------------------------------------------------------------------------------------------------------------------

**Был также рассчитан ожидаемый объем сырья от скважины для безубыточной стратегии:** 111.111 тыс.баррелей.

-------------------------------------------------------------------------------------------------------------------------------

Во всех 3 регионах среднее предсказанное количество сырья оказалось меньше, чем минимально требуется для безубыточной стратегии при бурении скважины. Но так как это очень простые расчеты, которые не всегда корректно отображают результат, нужна перепроверка результата техникой bootstrap.

-------------------------------------------------------------------------------------------------------------------------------

Также была рассчитана **суммарная прибыль в 3 регионах по топ-200 предсказанным скважинам** (по объему нефти):
1. Прибыль в первом регионе: 3320826043.14
2. Прибыль во втором регионе: 2415086696.68
3. Прибыль в третьем регионе: 2710349963.6

По этим расчетам самым прибыльным оказался первый регион. Но этих данных также недостаточно для принятия решения.

-------------------------------------------------------------------------------------------------------------------------------

**Была применена техника Bootstrap с 1000 выборок, чтобы найти распределение прибыли:**
в рамках этой техники была найдена средняя прибыль, 95%-й доверительный интервал и риск убытков по каждому из регионов.

**Средняя прибыль первого региона:** 600735244.261<br>
**95%-ый доверительный интервал первого региона:** 580991087.318, 620479401.203<br>
**Риск убытков первого региона:** 2%<br>
<br>
**Средняя прибыль второго региона:** 663958995.260<br>
**95%-ый доверительный интервал второго региона:** (648077906.015, 679840084.504)<br>
**Риск убытков второго региона:** 0.1%<br>
<br>
**Средняя прибыль третьего региона:** 597381047.900<br>
**95%-ый доверительный интервал третьего региона:** (577080076.079, 617682019.721)<br>
**Риск убытков третьего региона:** 2.5%<br>
<br>

📌--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------📌<br>
**В итоге для бурения я предлагаю выбрать второй регион, потому что там самый небольшой риск убытков - 0.1%, а также самая высокая прибыль.**<br>
📌--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------📌<br>